# Часть 5 Настраиваем модель

Из прошлой части у нас есть подготоленный набор данных, который дает неплохой результат.
Попробуем работу разных моделей, и разные параметры.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle
from functools import partial
from tqdm import tqdm

from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.base import clone

from catboost import CatBoostRegressor
import xgboost as xgb

Сделаем настройки ноутбука

In [2]:
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams['figure.figsize'] = 12, 8
RANDOM_STATE=43

## Прочитаем набор полученный на предыдущем шаге

In [3]:
df = pd.read_csv('data2.csv')
df_train = df[df.is_train==1]
print(f"{df.shape=}")

C:\Soft\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df.shape=(70896, 146)


## Определим функции для дальнейшего анализа

* `fit_and_score_model` - оценивает переданнух модель на кросс валидации, и выводит среднее значение метрики `mape`
* `make_submit` - делает предсказание модели на тестовых данныхЪ и сохраняет их в файл `predict.csv`
* `prepare_data` - подгаталивает тренировочный и тестовый набор (на основании переданного списка колонок)
* `get_top100_equipment_features` - список 100 фич из признака `equipment`

In [4]:
def mape(y_true, y_pred):
    """оценка MEAN ABSOLUTE PERCENTAGE ERROR, которая используется в соревновании"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    
def fit_and_score_model(model, X, y):
    """Оцениваем модель и данные на трех фолдах. Выводим средюю оценку, и оценки на фолдах"""
    
    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    cv_results = cross_val_score(
        model, X, y, cv=kf, n_jobs=-1, scoring=make_scorer(mape))
    
    model.fit(X_train,y)
    train_score = mape(y, model.predict(X_train))
    
    print("#"*100)
    print(model)
    print(f"train shape {X.shape}")
    print(f"{cat_cols=}")
    print(f"{num_cols=}")
    print("#"*100)
    print(f"Train result: {train_score}")
    print(f'CV result: {np.mean(cv_results)} ({cv_results})')
    print("#"*100)


def make_submit_file(model, X_test, file_name='predict.csv'):
    """запишем предсказание модели в файл, готовый к отправке на kaggle"""
    pred = np.round(model.predict(X_test))
    answer = pd.Series(pred, index=df[~df.is_train].sell_id, name='price')
    answer.to_csv(file_name, index_label='sell_id')


def prepare_data(categorical_columns: list, numerical_columns: list):
    """Вытащим требуемые данные из глобальной переменной df
    к категориальным применим dummy кодирование
    к числовым Шкалирование"""
    X = df.loc[:, categorical_columns+numerical_columns+['is_train']]

    X = pd.get_dummies(X, columns=categorical_columns)
    X[numerical_columns] = StandardScaler().fit_transform(X[numerical_columns])
    
    X_train = X[X.is_train == 1].drop('is_train', axis=1)
    X_test = X[X.is_train == 0].drop('is_train', axis=1)

    y = df[df.is_train].price

    return X_train, X_test, y

def get_top100_equipment_features():
    return [
        'engine-proof', 'tinted-glass', 'airbag-driver', 'aux', 'isofix',
        'electro-window-front', 'ashtray-and-cigarette-lighter',
        'airbag-passenger', 'computer', 'high-beam-assist', 'seat-transformation',
        'isofix-front', 'wheel-power', 'alarm', 'lock', 'door-sill-panel',
        'fabric-seats', 'electro-mirrors', 'airbag-rear-side',
        'electro-window-back', 'steel-wheels', 'ptf', '16-inch-wheels',
        'rain-sensor', 'airbag-side', 'audiosystem-cd', 'dark-interior',
        'cooling-box', 'condition', 'abs', 'power-child-locks-rear-doors',
        'front-seats-heat', 'wheel-configuration2', 'wheel-configuration1',
        '12v-socket', 'park-assist-r', 'cruise-control', 'asr', 'esp', 'usb',
        'light-sensor', 'drl', 'alloy-wheel-disks', 'tyre-pressure',
        'mirrors-heat', 'driver-seat-updown', 'wheel-heat', 'bluetooth', 'immo',
        'windscreen-heat', 'climate-control-2', 'adaptive-light', 'multi-wheel',
        'xenon', 'heated-wash-system', 'front-centre-armrest', 'body-mouldings',
        'combo-interior', 'rear-seats-heat', 'knee-airbag', 'hcc',
        'airbag-curtain', 'light-cleaner', 'leather-gear-stick', 'wheel-leather',
        'auto-mirrors', 'third-rear-headrest', 'roller-blind-for-rear-window',
        'spare-wheel', 'navigation', 'folding-front-passenger-seat',
        'programmed-block-heater', 'passenger-seat-updown', 'sport-seats',
        'front-seat-support', '18-inch-wheels', 'volume-sensor', 'hatch',
        'music-super', 'automatic-lighting-control', 'paint-metallic',
        'climate-control-1', 'roof-rails', '17-inch-wheels', 'sport-pedals', 'gbo',
        'park-assist-f', 'feedback-alarm', 'passenger-seat-electric', 'leather',
        'electro-trunk', 'driver-seat-memory', 'keyless-entry', 'rear-camera',
        'led-lights', '15-inch-wheels', 'audiosystem-tv', 'seats-7', 'auto-park',
        'apple-carplay'
    ]

## Подготовим данные

In [5]:
cat_cols = ['bodyType', 'brand', 'fuelType', 'color', 'model_name',
            'vehicleTransmission', 'vendor', 'ПТС', 'Владельцы', 'Привод', 'Руль'] 
num_cols = ['productionDate', 'numberOfDoors',
            'enginePower_log', 'mileage_log', 'diff_date_model_production'] + get_top100_equipment_features()

In [6]:
X_train, X_test, y = prepare_data(cat_cols, num_cols)

# Случайный лес

## Случайный лес из 1000 деревьев

In [9]:
model = RandomForestRegressor(1000, random_state=RANDOM_STATE, verbose=True)

In [10]:
%%time
fit_and_score_model(model, X_train, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 30.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


####################################################################################################
RandomForestRegressor(n_estimators=1000, random_state=43, verbose=True)
train shape (36210, 845)
cat_cols=['bodyType', 'brand', 'fuelType', 'color', 'model_name', 'vehicleTransmission', 'vendor', 'ПТС', 'Владельцы', 'Привод', 'Руль']
num_cols=['productionDate', 'numberOfDoors', 'enginePower_log', 'mileage_log', 'diff_date_model_production', 'engine-proof', 'tinted-glass', 'airbag-driver', 'aux', 'isofix', 'electro-window-front', 'ashtray-and-cigarette-lighter', 'airbag-passenger', 'computer', 'high-beam-assist', 'seat-transformation', 'isofix-front', 'wheel-power', 'alarm', 'lock', 'door-sill-panel', 'fabric-seats', 'electro-mirrors', 'airbag-rear-side', 'electro-window-back', 'steel-wheels', 'ptf', '16-inch-wheels', 'rain-sensor', 'airbag-side', 'audiosystem-cd', 'dark-interior', 'cooling-box', 'condition', 'abs', 'power-child-locks-rear-doors', 'front-seats-heat', 'wheel-configuration

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   14.6s finished


In [11]:
pd.Series(model.feature_importances_,
          index=X_test.columns).sort_values(ascending=False).iloc[:20]

enginePower_log              0.41
mileage_log                  0.34
productionDate               0.06
model_name_G-КЛАСС_AMG       0.03
model_name_G-КЛАСС           0.02
model_name_MAYBACH_S-КЛАСС   0.02
model_name_W188              0.01
fuelType_дизель              0.01
fuelType_бензин              0.01
model_name_V-КЛАСС           0.00
Привод_полный                0.00
Привод_задний                0.00
asr                          0.00
brand_MERCEDES               0.00
diff_date_model_production   0.00
numberOfDoors                0.00
model_name_CROWN             0.00
model_name_XC90              0.00
front-seats-heat             0.00
model_name_SLS_AMG           0.00
dtype: float64

In [12]:
make_submit_file(model, X_test, "5.1_random_forest_1000.csv") # 12.88

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   12.5s finished


## Случайный лес на 2000 деревьев

In [13]:
model = RandomForestRegressor(2000, random_state=RANDOM_STATE, verbose=True)

In [14]:
%%time
fit_and_score_model(model, X_train, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed: 57.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


####################################################################################################
RandomForestRegressor(n_estimators=2000, random_state=43, verbose=True)
train shape (36210, 845)
cat_cols=['bodyType', 'brand', 'fuelType', 'color', 'model_name', 'vehicleTransmission', 'vendor', 'ПТС', 'Владельцы', 'Привод', 'Руль']
num_cols=['productionDate', 'numberOfDoors', 'enginePower_log', 'mileage_log', 'diff_date_model_production', 'engine-proof', 'tinted-glass', 'airbag-driver', 'aux', 'isofix', 'electro-window-front', 'ashtray-and-cigarette-lighter', 'airbag-passenger', 'computer', 'high-beam-assist', 'seat-transformation', 'isofix-front', 'wheel-power', 'alarm', 'lock', 'door-sill-panel', 'fabric-seats', 'electro-mirrors', 'airbag-rear-side', 'electro-window-back', 'steel-wheels', 'ptf', '16-inch-wheels', 'rain-sensor', 'airbag-side', 'audiosystem-cd', 'dark-interior', 'cooling-box', 'condition', 'abs', 'power-child-locks-rear-doors', 'front-seats-heat', 'wheel-configuration

[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:   29.1s finished


In [15]:
make_submit_file(model, X_test, "5.2_random_forest_2000.csv") # 12.88

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:   24.9s finished


In [16]:
filename = '5.2.random_forest_2000.model'
pickle.dump(model, open(filename, 'wb'))

## Подбор параметров модели

Лучший результат показал случайный лес. Давайте попробуем как будет он себя вести, при изменени его деволтных параметров  
Хотел взять критерий MAE для  модели, но она очень долго тренирутеся, даже если поставить n_estimators=10

По результатам побора параметров - выяснилось что лучше всего случайный лес работает на стандартных параметрах.

In [7]:
rf_model=RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=100, verbose=True)

In [8]:
param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6, None],
}

In [9]:
CV_rf = GridSearchCV(rf_model, param_grid, cv=3)


In [10]:
%%time
CV_rf.fit(X_train, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   29.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   31.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   26.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

Wall time: 13min 57s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.9min finished


GridSearchCV(cv=3,
             estimator=RandomForestRegressor(random_state=43, verbose=True),
             param_grid={'max_depth': [4, 6, None],
                         'max_features': ['auto', 'sqrt', 'log2']})

In [11]:
CV_rf.best_params_

{'max_depth': None, 'max_features': 'auto'}

ну т.е. дефолтные параметры оказались лучшими.  
не очень интересно

## Выводы

####################################################################################################  
Train result: 4.9289989232811555   
CV result: 13.769852455608076 ([13.62834267 14.16325805 13.51795664])   
####################################################################################################   
Wall time: 1h 51min 16s

Наверно лучшая модель, показале неплохие результаты.  
Парметры лучше чем дефолтные найти не получилось.  
Чем больше деревьев тем больше точность, но добавлять более 2000 деревьев очень затратно по времени/ресурсам.

# CatBoost

In [16]:
def prepare_data_catboost(categorical_columns: list, numerical_columns: list):
    """Вытащим требуемые данные из глобальной переменной df
    к категориальным применим dummy кодирование
    к числовым Шкалирование"""
    X = df.loc[:, categorical_columns+numerical_columns+['is_train']]

    #X = pd.get_dummies(X, columns=categorical_columns)
    X[numerical_columns] = StandardScaler().fit_transform(X[numerical_columns])
    
    X_train = X[X.is_train == 1].drop('is_train', axis=1)
    X_test = X[X.is_train == 0].drop('is_train', axis=1)

    y = df[df.is_train].price

    return X_train, X_test, y

In [17]:
X_train_cat, X_test_cat, y = prepare_data_catboost(cat_cols, num_cols)

In [27]:
model = CatBoostRegressor(iterations=2000,
                          learning_rate=0.1,
                          random_seed=RANDOM_STATE,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          cat_features=cat_cols)

In [28]:
%%time
fit_and_score_model(model, X_train_cat, y)

0:	learn: 2.0248025	total: 72.4ms	remaining: 2m 24s
1:	learn: 1.8834781	total: 138ms	remaining: 2m 17s
2:	learn: 1.7303680	total: 215ms	remaining: 2m 23s
3:	learn: 1.6017261	total: 298ms	remaining: 2m 28s
4:	learn: 1.4888042	total: 372ms	remaining: 2m 28s
5:	learn: 1.3941541	total: 442ms	remaining: 2m 27s
6:	learn: 1.3001147	total: 508ms	remaining: 2m 24s
7:	learn: 1.2191658	total: 576ms	remaining: 2m 23s
8:	learn: 1.1318396	total: 643ms	remaining: 2m 22s
9:	learn: 1.0674587	total: 716ms	remaining: 2m 22s
10:	learn: 1.0041237	total: 790ms	remaining: 2m 22s
11:	learn: 0.9500556	total: 861ms	remaining: 2m 22s
12:	learn: 0.8926039	total: 932ms	remaining: 2m 22s
13:	learn: 0.8495864	total: 1.01s	remaining: 2m 22s
14:	learn: 0.8062841	total: 1.07s	remaining: 2m 21s
15:	learn: 0.7537520	total: 1.14s	remaining: 2m 20s
16:	learn: 0.7212745	total: 1.21s	remaining: 2m 20s
17:	learn: 0.6937995	total: 1.27s	remaining: 2m 20s
18:	learn: 0.6549367	total: 1.34s	remaining: 2m 19s
19:	learn: 0.6151454	

159:	learn: 0.2023117	total: 11.6s	remaining: 2m 13s
160:	learn: 0.2022967	total: 11.7s	remaining: 2m 13s
161:	learn: 0.2020029	total: 11.7s	remaining: 2m 13s
162:	learn: 0.2014457	total: 11.8s	remaining: 2m 13s
163:	learn: 0.2009427	total: 11.9s	remaining: 2m 12s
164:	learn: 0.2004150	total: 11.9s	remaining: 2m 12s
165:	learn: 0.2004706	total: 12s	remaining: 2m 12s
166:	learn: 0.1985411	total: 12.1s	remaining: 2m 12s
167:	learn: 0.1984903	total: 12.2s	remaining: 2m 12s
168:	learn: 0.1986562	total: 12.2s	remaining: 2m 12s
169:	learn: 0.1982607	total: 12.3s	remaining: 2m 12s
170:	learn: 0.1980947	total: 12.4s	remaining: 2m 12s
171:	learn: 0.1976607	total: 12.5s	remaining: 2m 12s
172:	learn: 0.1968730	total: 12.5s	remaining: 2m 12s
173:	learn: 0.1966031	total: 12.6s	remaining: 2m 12s
174:	learn: 0.1966368	total: 12.7s	remaining: 2m 12s
175:	learn: 0.1963746	total: 12.8s	remaining: 2m 12s
176:	learn: 0.1963958	total: 12.8s	remaining: 2m 12s
177:	learn: 0.1964118	total: 12.9s	remaining: 2m

317:	learn: 0.1708311	total: 25.4s	remaining: 2m 14s
318:	learn: 0.1706869	total: 25.5s	remaining: 2m 14s
319:	learn: 0.1706762	total: 25.5s	remaining: 2m 14s
320:	learn: 0.1701058	total: 25.6s	remaining: 2m 13s
321:	learn: 0.1699280	total: 25.7s	remaining: 2m 13s
322:	learn: 0.1699315	total: 25.8s	remaining: 2m 13s
323:	learn: 0.1695676	total: 25.8s	remaining: 2m 13s
324:	learn: 0.1695393	total: 25.9s	remaining: 2m 13s
325:	learn: 0.1693352	total: 26s	remaining: 2m 13s
326:	learn: 0.1692444	total: 26.1s	remaining: 2m 13s
327:	learn: 0.1689630	total: 26.1s	remaining: 2m 13s
328:	learn: 0.1689952	total: 26.2s	remaining: 2m 13s
329:	learn: 0.1688742	total: 26.3s	remaining: 2m 13s
330:	learn: 0.1687827	total: 26.4s	remaining: 2m 12s
331:	learn: 0.1686859	total: 26.4s	remaining: 2m 12s
332:	learn: 0.1684925	total: 26.5s	remaining: 2m 12s
333:	learn: 0.1684580	total: 26.6s	remaining: 2m 12s
334:	learn: 0.1684163	total: 26.7s	remaining: 2m 12s
335:	learn: 0.1683635	total: 26.7s	remaining: 2m

476:	learn: 0.1563323	total: 39.7s	remaining: 2m 6s
477:	learn: 0.1562734	total: 39.8s	remaining: 2m 6s
478:	learn: 0.1561217	total: 39.9s	remaining: 2m 6s
479:	learn: 0.1560441	total: 40.1s	remaining: 2m 6s
480:	learn: 0.1560971	total: 40.2s	remaining: 2m 6s
481:	learn: 0.1559785	total: 40.3s	remaining: 2m 6s
482:	learn: 0.1558801	total: 40.4s	remaining: 2m 6s
483:	learn: 0.1557283	total: 40.5s	remaining: 2m 6s
484:	learn: 0.1557357	total: 40.6s	remaining: 2m 6s
485:	learn: 0.1556511	total: 40.7s	remaining: 2m 6s
486:	learn: 0.1556188	total: 40.8s	remaining: 2m 6s
487:	learn: 0.1556291	total: 40.9s	remaining: 2m 6s
488:	learn: 0.1555881	total: 41s	remaining: 2m 6s
489:	learn: 0.1555984	total: 41.1s	remaining: 2m 6s
490:	learn: 0.1555965	total: 41.2s	remaining: 2m 6s
491:	learn: 0.1552600	total: 41.3s	remaining: 2m 6s
492:	learn: 0.1552422	total: 41.4s	remaining: 2m 6s
493:	learn: 0.1552955	total: 41.5s	remaining: 2m 6s
494:	learn: 0.1550689	total: 41.6s	remaining: 2m 6s
495:	learn: 0.

635:	learn: 0.1469460	total: 53.2s	remaining: 1m 54s
636:	learn: 0.1469506	total: 53.2s	remaining: 1m 53s
637:	learn: 0.1469277	total: 53.3s	remaining: 1m 53s
638:	learn: 0.1469090	total: 53.4s	remaining: 1m 53s
639:	learn: 0.1468886	total: 53.5s	remaining: 1m 53s
640:	learn: 0.1469028	total: 53.6s	remaining: 1m 53s
641:	learn: 0.1467654	total: 53.7s	remaining: 1m 53s
642:	learn: 0.1465245	total: 53.8s	remaining: 1m 53s
643:	learn: 0.1465202	total: 54s	remaining: 1m 53s
644:	learn: 0.1465099	total: 54.1s	remaining: 1m 53s
645:	learn: 0.1464694	total: 54.2s	remaining: 1m 53s
646:	learn: 0.1464249	total: 54.3s	remaining: 1m 53s
647:	learn: 0.1463770	total: 54.3s	remaining: 1m 53s
648:	learn: 0.1462834	total: 54.4s	remaining: 1m 53s
649:	learn: 0.1462385	total: 54.5s	remaining: 1m 53s
650:	learn: 0.1462082	total: 54.6s	remaining: 1m 53s
651:	learn: 0.1461527	total: 54.7s	remaining: 1m 53s
652:	learn: 0.1460792	total: 54.8s	remaining: 1m 53s
653:	learn: 0.1460566	total: 54.9s	remaining: 1m

...

1247:	learn: 0.1287300	total: 1m 50s	remaining: 1m 6s
1248:	learn: 0.1286543	total: 1m 50s	remaining: 1m 6s
1249:	learn: 0.1286369	total: 1m 50s	remaining: 1m 6s
1250:	learn: 0.1286458	total: 1m 51s	remaining: 1m 6s
1251:	learn: 0.1286544	total: 1m 51s	remaining: 1m 6s
1252:	learn: 0.1285534	total: 1m 51s	remaining: 1m 6s
1253:	learn: 0.1285126	total: 1m 51s	remaining: 1m 6s
1254:	learn: 0.1285160	total: 1m 51s	remaining: 1m 6s
1255:	learn: 0.1285074	total: 1m 51s	remaining: 1m 6s
1256:	learn: 0.1284878	total: 1m 51s	remaining: 1m 5s
1257:	learn: 0.1284567	total: 1m 51s	remaining: 1m 5s
1258:	learn: 0.1284034	total: 1m 51s	remaining: 1m 5s
1259:	learn: 0.1283892	total: 1m 51s	remaining: 1m 5s
1260:	learn: 0.1283825	total: 1m 51s	remaining: 1m 5s
1261:	learn: 0.1283663	total: 1m 52s	remaining: 1m 5s
1262:	learn: 0.1283836	total: 1m 52s	remaining: 1m 5s
1263:	learn: 0.1282667	total: 1m 52s	remaining: 1m 5s
1264:	learn: 0.1282782	total: 1m 52s	remaining: 1m 5s
1265:	learn: 0.1282602	total

1403:	learn: 0.1255623	total: 2m 5s	remaining: 53.1s
1404:	learn: 0.1255652	total: 2m 5s	remaining: 53s
1405:	learn: 0.1255711	total: 2m 5s	remaining: 52.9s
1406:	learn: 0.1255197	total: 2m 5s	remaining: 52.8s
1407:	learn: 0.1255457	total: 2m 5s	remaining: 52.7s
1408:	learn: 0.1255800	total: 2m 5s	remaining: 52.7s
1409:	learn: 0.1255746	total: 2m 5s	remaining: 52.6s
1410:	learn: 0.1255291	total: 2m 5s	remaining: 52.5s
1411:	learn: 0.1255097	total: 2m 5s	remaining: 52.4s
1412:	learn: 0.1255376	total: 2m 5s	remaining: 52.3s
1413:	learn: 0.1254714	total: 2m 6s	remaining: 52.2s
1414:	learn: 0.1254729	total: 2m 6s	remaining: 52.1s
1415:	learn: 0.1254393	total: 2m 6s	remaining: 52.1s
1416:	learn: 0.1254501	total: 2m 6s	remaining: 52s
1417:	learn: 0.1254570	total: 2m 6s	remaining: 51.9s
1418:	learn: 0.1254493	total: 2m 6s	remaining: 51.8s
1419:	learn: 0.1254375	total: 2m 6s	remaining: 51.7s
1420:	learn: 0.1253987	total: 2m 6s	remaining: 51.6s
1421:	learn: 0.1253916	total: 2m 6s	remaining: 51.

1558:	learn: 0.1233730	total: 2m 20s	remaining: 39.9s
1559:	learn: 0.1233740	total: 2m 21s	remaining: 39.8s
1560:	learn: 0.1232997	total: 2m 21s	remaining: 39.7s
1561:	learn: 0.1232923	total: 2m 21s	remaining: 39.6s
1562:	learn: 0.1232808	total: 2m 21s	remaining: 39.5s
1563:	learn: 0.1232921	total: 2m 21s	remaining: 39.4s
1564:	learn: 0.1232668	total: 2m 21s	remaining: 39.4s
1565:	learn: 0.1232556	total: 2m 21s	remaining: 39.3s
1566:	learn: 0.1232354	total: 2m 21s	remaining: 39.2s
1567:	learn: 0.1231910	total: 2m 21s	remaining: 39.1s
1568:	learn: 0.1232052	total: 2m 21s	remaining: 39s
1569:	learn: 0.1231514	total: 2m 22s	remaining: 38.9s
1570:	learn: 0.1231377	total: 2m 22s	remaining: 38.8s
1571:	learn: 0.1230814	total: 2m 22s	remaining: 38.7s
1572:	learn: 0.1230761	total: 2m 22s	remaining: 38.6s
1573:	learn: 0.1231012	total: 2m 22s	remaining: 38.6s
1574:	learn: 0.1230964	total: 2m 22s	remaining: 38.5s
1575:	learn: 0.1230956	total: 2m 22s	remaining: 38.4s
1576:	learn: 0.1230909	total: 

1712:	learn: 0.1212789	total: 2m 36s	remaining: 26.2s
1713:	learn: 0.1212660	total: 2m 36s	remaining: 26.1s
1714:	learn: 0.1212309	total: 2m 36s	remaining: 26s
1715:	learn: 0.1212360	total: 2m 36s	remaining: 25.9s
1716:	learn: 0.1212374	total: 2m 36s	remaining: 25.8s
1717:	learn: 0.1212320	total: 2m 36s	remaining: 25.7s
1718:	learn: 0.1212271	total: 2m 36s	remaining: 25.6s
1719:	learn: 0.1212167	total: 2m 36s	remaining: 25.5s
1720:	learn: 0.1212097	total: 2m 36s	remaining: 25.4s
1721:	learn: 0.1211733	total: 2m 36s	remaining: 25.3s
1722:	learn: 0.1210800	total: 2m 37s	remaining: 25.3s
1723:	learn: 0.1210873	total: 2m 37s	remaining: 25.2s
1724:	learn: 0.1210767	total: 2m 37s	remaining: 25.1s
1725:	learn: 0.1210367	total: 2m 37s	remaining: 25s
1726:	learn: 0.1209889	total: 2m 37s	remaining: 24.9s
1727:	learn: 0.1209609	total: 2m 37s	remaining: 24.8s
1728:	learn: 0.1209452	total: 2m 37s	remaining: 24.7s
1729:	learn: 0.1209498	total: 2m 37s	remaining: 24.6s
1730:	learn: 0.1209480	total: 2m

1865:	learn: 0.1193271	total: 2m 49s	remaining: 12.2s
1866:	learn: 0.1193165	total: 2m 49s	remaining: 12.1s
1867:	learn: 0.1193172	total: 2m 50s	remaining: 12s
1868:	learn: 0.1192487	total: 2m 50s	remaining: 11.9s
1869:	learn: 0.1192514	total: 2m 50s	remaining: 11.8s
1870:	learn: 0.1192369	total: 2m 50s	remaining: 11.7s
1871:	learn: 0.1192196	total: 2m 50s	remaining: 11.7s
1872:	learn: 0.1192290	total: 2m 50s	remaining: 11.6s
1873:	learn: 0.1192093	total: 2m 50s	remaining: 11.5s
1874:	learn: 0.1191899	total: 2m 50s	remaining: 11.4s
1875:	learn: 0.1191698	total: 2m 50s	remaining: 11.3s
1876:	learn: 0.1191633	total: 2m 50s	remaining: 11.2s
1877:	learn: 0.1191642	total: 2m 51s	remaining: 11.1s
1878:	learn: 0.1191383	total: 2m 51s	remaining: 11s
1879:	learn: 0.1191420	total: 2m 51s	remaining: 10.9s
1880:	learn: 0.1191042	total: 2m 51s	remaining: 10.8s
1881:	learn: 0.1191001	total: 2m 51s	remaining: 10.7s
1882:	learn: 0.1190994	total: 2m 51s	remaining: 10.7s
1883:	learn: 0.1190330	total: 2m

In [29]:
pd.Series(model.get_feature_importance(),
          index=model.feature_names_).sort_values(ascending=False)

enginePower_log        32.93
mileage_log            21.55
productionDate         11.43
brand                   9.62
fuelType                3.42
                        ... 
spare-wheel             0.00
gbo                     0.00
light-sensor            0.00
front-centre-armrest    0.00
15-inch-wheels          0.00
Length: 116, dtype: float64

In [30]:
make_submit_file(model, X_test_cat, "5.3_catboost.csv") # 18.83

In [31]:
filename = '5.3.catboost.model'
pickle.dump(model, open(filename, 'wb'))

## Выводы

####################################################################################################  
Train result: 11.842847961363898  
CV result: 14.054763710501483 ([14.47649167 14.38259282 13.30520664])  
####################################################################################################  
Wall time: 11min 20s  

Catboost плохо себя показал, или может просто я не умею его готовить.  
Чем ниже learning_rate тем лучше результат.

# XGBOOST

In [36]:
X_train, X_test, y = prepare_data(cat_cols, num_cols)

In [43]:
model = xgb.XGBRegressor(objective='reg:squarederror', 
                          colsample_bytree=0.5,
                          learning_rate=0.1, 
                          max_depth=12, 
                          alpha=1,
                          n_estimators=2000, 
                          silent=1)


In [44]:
%%time
fit_and_score_model(model, X_train, y)

[13:05:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


####################################################################################################
XGBRegressor(alpha=1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=12,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, silent=1,
            

In [51]:
make_submit_file(model, X_test, "5.4_xgboost.csv") # 13.27

In [50]:
filename = '5.4.xgboost.model'
pickle.dump(model, open(filename, 'wb'))

## Выводы

XGboost справился неплохо.  
Лучше Catboost но хуже случайного леса.

# Стекинг

Стекинг (stacking) — агрегация ответов моделей машинного обучения при помощи ещё одной модели машинного обучения. Подход использует понятие базовых моделей, каждая из которых обучается независимо от остальных, и мета-модели, которая использует предсказания базовых моделей как признаки.

По результатам стекинг - не улучшил модель. Может я выбрал неудачные модели для ансамбля.  
Результат и на валидации, и на лидерборде оказался хуже, чем применение одного случайного леса.  

In [11]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    X_meta_train = np.zeros((len(y_train)), dtype=np.float32)

    splits = cv.split(X_train)
    
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict(X_test)
    
    return X_meta_train, X_meta_test

def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ]).reshape(-1,len(features), order='F')

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ]).reshape(-1,len(features), order='F')
    
    return stacked_features_train, stacked_features_test



In [12]:
cv = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

In [13]:
%%time
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=RANDOM_STATE),
    ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_STATE),
    Ridge(random_state=RANDOM_STATE),
    GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
    CatBoostRegressor(random_state=RANDOM_STATE)
], X_train.values, X_test.values, y.values, cv)

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [28:41<06:05, 365.89s/it]

Learning rate set to 0.06879
0:	learn: 2168507.9405378	total: 186ms	remaining: 3m 5s
1:	learn: 2062797.6859961	total: 199ms	remaining: 1m 39s
2:	learn: 1963710.9747727	total: 216ms	remaining: 1m 11s
3:	learn: 1873458.6552986	total: 229ms	remaining: 57.1s
4:	learn: 1789447.0264457	total: 242ms	remaining: 48.2s
5:	learn: 1711745.1497783	total: 257ms	remaining: 42.5s
6:	learn: 1634544.2279880	total: 280ms	remaining: 39.8s
7:	learn: 1565346.2213272	total: 293ms	remaining: 36.3s
8:	learn: 1503003.0533334	total: 306ms	remaining: 33.7s
9:	learn: 1441617.3638499	total: 318ms	remaining: 31.5s
10:	learn: 1387241.3592203	total: 332ms	remaining: 29.8s
11:	learn: 1333018.2705210	total: 345ms	remaining: 28.4s
12:	learn: 1283084.2091846	total: 358ms	remaining: 27.2s
13:	learn: 1235567.3849376	total: 372ms	remaining: 26.2s
14:	learn: 1194836.9031313	total: 390ms	remaining: 25.6s
15:	learn: 1153106.6240734	total: 405ms	remaining: 24.9s
16:	learn: 1115934.5412226	total: 420ms	remaining: 24.3s
17:	learn:

150:	learn: 497454.5606200	total: 2.2s	remaining: 12.4s
151:	learn: 496251.9004379	total: 2.21s	remaining: 12.3s
152:	learn: 496027.9550870	total: 2.22s	remaining: 12.3s
153:	learn: 495189.8679911	total: 2.24s	remaining: 12.3s
154:	learn: 494976.4433126	total: 2.25s	remaining: 12.3s
155:	learn: 494022.7623968	total: 2.26s	remaining: 12.2s
156:	learn: 493792.2241409	total: 2.27s	remaining: 12.2s
157:	learn: 492937.2545426	total: 2.29s	remaining: 12.2s
158:	learn: 492716.5434926	total: 2.3s	remaining: 12.2s
159:	learn: 491634.7965834	total: 2.31s	remaining: 12.1s
160:	learn: 490222.5977920	total: 2.32s	remaining: 12.1s
161:	learn: 490009.7365147	total: 2.33s	remaining: 12.1s
162:	learn: 488486.4864975	total: 2.35s	remaining: 12.1s
163:	learn: 487133.4066870	total: 2.36s	remaining: 12s
164:	learn: 486649.2074172	total: 2.38s	remaining: 12s
165:	learn: 485604.9256594	total: 2.39s	remaining: 12s
166:	learn: 484948.1749940	total: 2.4s	remaining: 12s
167:	learn: 484765.0806900	total: 2.41s	re

304:	learn: 403284.8636052	total: 4.2s	remaining: 9.57s
305:	learn: 402860.8213107	total: 4.21s	remaining: 9.55s
306:	learn: 402499.2816932	total: 4.22s	remaining: 9.54s
307:	learn: 401905.7285788	total: 4.24s	remaining: 9.52s
308:	learn: 401239.8122774	total: 4.25s	remaining: 9.5s
309:	learn: 400807.2772797	total: 4.26s	remaining: 9.48s
310:	learn: 400730.7354521	total: 4.29s	remaining: 9.5s
311:	learn: 400647.7906050	total: 4.3s	remaining: 9.48s
312:	learn: 399968.2536145	total: 4.31s	remaining: 9.47s
313:	learn: 399547.9869040	total: 4.33s	remaining: 9.45s
314:	learn: 399395.8048404	total: 4.34s	remaining: 9.43s
315:	learn: 398925.2654183	total: 4.35s	remaining: 9.42s
316:	learn: 398502.0371464	total: 4.37s	remaining: 9.42s
317:	learn: 397891.6539261	total: 4.38s	remaining: 9.4s
318:	learn: 397586.8514816	total: 4.4s	remaining: 9.39s
319:	learn: 397513.2060947	total: 4.41s	remaining: 9.37s
320:	learn: 396944.5482228	total: 4.42s	remaining: 9.35s
321:	learn: 396617.1614378	total: 4.4

453:	learn: 351671.9004995	total: 6.15s	remaining: 7.4s
454:	learn: 351633.2692983	total: 6.16s	remaining: 7.38s
455:	learn: 351301.8124453	total: 6.18s	remaining: 7.37s
456:	learn: 351241.4027560	total: 6.19s	remaining: 7.35s
457:	learn: 351199.7949692	total: 6.2s	remaining: 7.34s
458:	learn: 350848.7968958	total: 6.21s	remaining: 7.32s
459:	learn: 350811.3260041	total: 6.23s	remaining: 7.31s
460:	learn: 350611.3772151	total: 6.24s	remaining: 7.29s
461:	learn: 350309.8812444	total: 6.25s	remaining: 7.28s
462:	learn: 350149.8109363	total: 6.26s	remaining: 7.26s
463:	learn: 349893.7998881	total: 6.28s	remaining: 7.25s
464:	learn: 349386.6201296	total: 6.29s	remaining: 7.24s
465:	learn: 349141.9976538	total: 6.3s	remaining: 7.22s
466:	learn: 348928.5223540	total: 6.32s	remaining: 7.21s
467:	learn: 348607.9576017	total: 6.33s	remaining: 7.2s
468:	learn: 348442.5524036	total: 6.35s	remaining: 7.19s
469:	learn: 348235.4141227	total: 6.36s	remaining: 7.17s
470:	learn: 347695.1390828	total: 6

...

646:	learn: 314934.0567274	total: 8.75s	remaining: 4.78s
647:	learn: 314723.2042193	total: 8.77s	remaining: 4.76s
648:	learn: 314701.8909369	total: 8.78s	remaining: 4.75s
649:	learn: 314561.8772592	total: 8.8s	remaining: 4.74s
650:	learn: 314369.8596749	total: 8.81s	remaining: 4.72s
651:	learn: 314289.8680001	total: 8.82s	remaining: 4.71s
652:	learn: 314210.7293263	total: 8.84s	remaining: 4.7s
653:	learn: 314082.4503798	total: 8.85s	remaining: 4.68s
654:	learn: 313858.7743142	total: 8.86s	remaining: 4.67s
655:	learn: 313686.0007972	total: 8.87s	remaining: 4.65s
656:	learn: 313665.0872225	total: 8.88s	remaining: 4.64s
657:	learn: 313571.3838643	total: 8.9s	remaining: 4.62s
658:	learn: 313374.3807240	total: 8.91s	remaining: 4.61s
659:	learn: 312772.6246889	total: 8.93s	remaining: 4.6s
660:	learn: 312657.3987516	total: 8.94s	remaining: 4.59s
661:	learn: 312523.6629403	total: 8.95s	remaining: 4.57s
662:	learn: 312299.7451508	total: 8.97s	remaining: 4.56s
663:	learn: 311972.0181788	total: 8

798:	learn: 292108.0310774	total: 10.8s	remaining: 2.72s
799:	learn: 292084.5996036	total: 10.8s	remaining: 2.71s
800:	learn: 292012.7206750	total: 10.8s	remaining: 2.69s
801:	learn: 291838.5107562	total: 10.9s	remaining: 2.68s
802:	learn: 291719.6742112	total: 10.9s	remaining: 2.67s
803:	learn: 291632.0583802	total: 10.9s	remaining: 2.65s
804:	learn: 291505.3917016	total: 10.9s	remaining: 2.64s
805:	learn: 291490.4116990	total: 10.9s	remaining: 2.63s
806:	learn: 291475.9935885	total: 10.9s	remaining: 2.61s
807:	learn: 291358.1848643	total: 10.9s	remaining: 2.6s
808:	learn: 291297.0715200	total: 10.9s	remaining: 2.58s
809:	learn: 291092.6822363	total: 11s	remaining: 2.57s
810:	learn: 290944.1739818	total: 11s	remaining: 2.56s
811:	learn: 290703.1245572	total: 11s	remaining: 2.54s
812:	learn: 290533.4474231	total: 11s	remaining: 2.53s
813:	learn: 290337.1108178	total: 11s	remaining: 2.52s
814:	learn: 290128.8023062	total: 11s	remaining: 2.5s
815:	learn: 289788.5106071	total: 11.1s	remai

955:	learn: 275113.3660073	total: 12.9s	remaining: 595ms
956:	learn: 274818.9035368	total: 12.9s	remaining: 581ms
957:	learn: 274807.9960365	total: 13s	remaining: 568ms
958:	learn: 274747.1579213	total: 13s	remaining: 554ms
959:	learn: 274568.2905592	total: 13s	remaining: 541ms
960:	learn: 274415.0880537	total: 13s	remaining: 527ms
961:	learn: 274311.3131627	total: 13s	remaining: 514ms
962:	learn: 274286.5608168	total: 13s	remaining: 500ms
963:	learn: 274154.9061220	total: 13s	remaining: 487ms
964:	learn: 274097.2496438	total: 13s	remaining: 473ms
965:	learn: 274000.1461157	total: 13.1s	remaining: 460ms
966:	learn: 273893.9836987	total: 13.1s	remaining: 446ms
967:	learn: 273883.2865687	total: 13.1s	remaining: 433ms
968:	learn: 273770.0533917	total: 13.1s	remaining: 419ms
969:	learn: 273756.0185262	total: 13.1s	remaining: 406ms
970:	learn: 273722.5287004	total: 13.1s	remaining: 392ms
971:	learn: 273574.1222408	total: 13.1s	remaining: 379ms
972:	learn: 273464.5853100	total: 13.2s	remaini

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [30:23<00:00, 364.64s/it]

Wall time: 30min 23s


In [14]:
super_model = RandomForestRegressor(random_state=RANDOM_STATE)
super_model.fit(stacked_features_train, y)

RandomForestRegressor(random_state=43)

In [15]:
mape(y, super_model.predict(stacked_features_train))

5.155586473043821

In [17]:
make_submit_file(super_model, stacked_features_test, "stacking_3.csv") # 14.32

Плохой результат получался, увеличим количество split до 10

In [18]:
cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

In [19]:
%%time
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=RANDOM_STATE),
    Ridge(random_state=RANDOM_STATE),
    GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
    CatBoostRegressor(random_state=RANDOM_STATE)
], X_train.values, X_test.values, y.values, cv)

 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [41:30<15:48, 948.01s/it]

Learning rate set to 0.072565
0:	learn: 2243910.9777324	total: 14.2ms	remaining: 14.2s
1:	learn: 2127340.6099715	total: 27ms	remaining: 13.5s
2:	learn: 2026473.4593176	total: 40.3ms	remaining: 13.4s
3:	learn: 1928534.4231416	total: 52.9ms	remaining: 13.2s
4:	learn: 1838292.5722514	total: 66.7ms	remaining: 13.3s
5:	learn: 1755496.7753477	total: 78.9ms	remaining: 13.1s
6:	learn: 1679137.9491829	total: 91.6ms	remaining: 13s
7:	learn: 1608985.0010856	total: 104ms	remaining: 12.9s
8:	learn: 1539506.7326033	total: 117ms	remaining: 12.8s
9:	learn: 1474889.8739987	total: 129ms	remaining: 12.7s
10:	learn: 1417801.7614110	total: 141ms	remaining: 12.7s
11:	learn: 1362375.3041449	total: 154ms	remaining: 12.6s
12:	learn: 1311372.7318139	total: 167ms	remaining: 12.7s
13:	learn: 1263098.9959274	total: 180ms	remaining: 12.7s
14:	learn: 1221315.3561112	total: 194ms	remaining: 12.7s
15:	learn: 1182066.4862316	total: 207ms	remaining: 12.8s
16:	learn: 1145479.3218495	total: 221ms	remaining: 12.8s
17:	lear

146:	learn: 524529.8170335	total: 1.79s	remaining: 10.4s
147:	learn: 524165.3373653	total: 1.8s	remaining: 10.4s
148:	learn: 523809.3522982	total: 1.81s	remaining: 10.3s
149:	learn: 522180.4101295	total: 1.82s	remaining: 10.3s
150:	learn: 521894.8904696	total: 1.83s	remaining: 10.3s
151:	learn: 519469.9991743	total: 1.84s	remaining: 10.3s
152:	learn: 519140.7531518	total: 1.86s	remaining: 10.3s
153:	learn: 517807.0829922	total: 1.87s	remaining: 10.3s
154:	learn: 517446.1240647	total: 1.88s	remaining: 10.2s
155:	learn: 517189.1644289	total: 1.89s	remaining: 10.2s
156:	learn: 515518.1070241	total: 1.9s	remaining: 10.2s
157:	learn: 514397.2158905	total: 1.92s	remaining: 10.2s
158:	learn: 512831.6715040	total: 1.93s	remaining: 10.2s
159:	learn: 512599.2434736	total: 1.94s	remaining: 10.2s
160:	learn: 511466.2776771	total: 1.95s	remaining: 10.2s
161:	learn: 511206.5174743	total: 1.96s	remaining: 10.2s
162:	learn: 509698.4449924	total: 1.98s	remaining: 10.1s
163:	learn: 508640.5472636	total:

299:	learn: 418146.3949471	total: 3.61s	remaining: 8.42s
300:	learn: 417797.4644762	total: 3.62s	remaining: 8.4s
301:	learn: 417717.9020204	total: 3.63s	remaining: 8.39s
302:	learn: 417598.1039584	total: 3.64s	remaining: 8.38s
303:	learn: 417520.9707554	total: 3.65s	remaining: 8.36s
304:	learn: 416955.2000265	total: 3.67s	remaining: 8.35s
305:	learn: 416239.8550535	total: 3.68s	remaining: 8.34s
306:	learn: 415679.1343561	total: 3.69s	remaining: 8.33s
307:	learn: 415549.6659385	total: 3.7s	remaining: 8.32s
308:	learn: 415474.3729996	total: 3.71s	remaining: 8.31s
309:	learn: 415125.9775612	total: 3.73s	remaining: 8.3s
310:	learn: 414490.9140966	total: 3.74s	remaining: 8.28s
311:	learn: 414064.9913913	total: 3.75s	remaining: 8.27s
312:	learn: 413301.0260762	total: 3.76s	remaining: 8.26s
313:	learn: 411942.2749805	total: 3.78s	remaining: 8.25s
314:	learn: 411400.5098015	total: 3.79s	remaining: 8.24s
315:	learn: 411289.3379391	total: 3.8s	remaining: 8.23s
316:	learn: 410528.3015189	total: 3

445:	learn: 363473.1016969	total: 5.39s	remaining: 6.7s
446:	learn: 363429.2552723	total: 5.4s	remaining: 6.68s
447:	learn: 362947.8993780	total: 5.42s	remaining: 6.67s
448:	learn: 362876.0782122	total: 5.43s	remaining: 6.66s
449:	learn: 362615.3245699	total: 5.44s	remaining: 6.65s
450:	learn: 362240.5259815	total: 5.45s	remaining: 6.64s
451:	learn: 361460.5101793	total: 5.46s	remaining: 6.63s
452:	learn: 361228.9394825	total: 5.48s	remaining: 6.61s
453:	learn: 361164.7080515	total: 5.49s	remaining: 6.6s
454:	learn: 361122.0688256	total: 5.5s	remaining: 6.59s
455:	learn: 360789.5562339	total: 5.51s	remaining: 6.58s
456:	learn: 360512.5010552	total: 5.53s	remaining: 6.57s
457:	learn: 360291.1226013	total: 5.54s	remaining: 6.55s
458:	learn: 359655.9843786	total: 5.55s	remaining: 6.54s
459:	learn: 359324.1871463	total: 5.56s	remaining: 6.53s
460:	learn: 359282.5003957	total: 5.57s	remaining: 6.52s
461:	learn: 358793.3468670	total: 5.59s	remaining: 6.5s
462:	learn: 358737.6580771	total: 5.

...

579:	learn: 326739.4561031	total: 7.64s	remaining: 5.53s
580:	learn: 326620.8930948	total: 7.65s	remaining: 5.52s
581:	learn: 326343.0833336	total: 7.66s	remaining: 5.5s
582:	learn: 326090.6109432	total: 7.67s	remaining: 5.49s
583:	learn: 325937.8796217	total: 7.69s	remaining: 5.48s
584:	learn: 325444.8644606	total: 7.7s	remaining: 5.46s
585:	learn: 325183.1632959	total: 7.71s	remaining: 5.45s
586:	learn: 325039.6510866	total: 7.73s	remaining: 5.44s
587:	learn: 325003.1983413	total: 7.74s	remaining: 5.42s
588:	learn: 324655.8566113	total: 7.75s	remaining: 5.41s
589:	learn: 324549.5740062	total: 7.76s	remaining: 5.39s
590:	learn: 324455.4250641	total: 7.78s	remaining: 5.38s
591:	learn: 324146.2739448	total: 7.79s	remaining: 5.37s
592:	learn: 323901.6474941	total: 7.8s	remaining: 5.36s
593:	learn: 323875.7972648	total: 7.82s	remaining: 5.34s
594:	learn: 323647.3789929	total: 7.83s	remaining: 5.33s
595:	learn: 323444.2318001	total: 7.84s	remaining: 5.32s
596:	learn: 323418.8946570	total: 

733:	learn: 301254.1691486	total: 9.62s	remaining: 3.48s
734:	learn: 301027.3059263	total: 9.63s	remaining: 3.47s
735:	learn: 300838.3198563	total: 9.64s	remaining: 3.46s
736:	learn: 300788.9668881	total: 9.66s	remaining: 3.45s
737:	learn: 300749.7471796	total: 9.67s	remaining: 3.43s
738:	learn: 300329.6105032	total: 9.68s	remaining: 3.42s
739:	learn: 300128.0221288	total: 9.7s	remaining: 3.41s
740:	learn: 300085.1271025	total: 9.71s	remaining: 3.39s
741:	learn: 300054.0923554	total: 9.72s	remaining: 3.38s
742:	learn: 299979.8735906	total: 9.74s	remaining: 3.37s
743:	learn: 299864.1730410	total: 9.75s	remaining: 3.35s
744:	learn: 299708.7484985	total: 9.76s	remaining: 3.34s
745:	learn: 299586.8453779	total: 9.78s	remaining: 3.33s
746:	learn: 299569.1638171	total: 9.79s	remaining: 3.31s
747:	learn: 299439.1348401	total: 9.8s	remaining: 3.3s
748:	learn: 299260.5474316	total: 9.82s	remaining: 3.29s
749:	learn: 299144.4605875	total: 9.83s	remaining: 3.28s
750:	learn: 299024.9488743	total: 

880:	learn: 282592.6954233	total: 11.5s	remaining: 1.56s
881:	learn: 282580.4051496	total: 11.5s	remaining: 1.54s
882:	learn: 282543.6652023	total: 11.6s	remaining: 1.53s
883:	learn: 282411.7008237	total: 11.6s	remaining: 1.52s
884:	learn: 282312.1648104	total: 11.6s	remaining: 1.5s
885:	learn: 282148.9822533	total: 11.6s	remaining: 1.49s
886:	learn: 282105.1384503	total: 11.6s	remaining: 1.48s
887:	learn: 282018.5959972	total: 11.6s	remaining: 1.47s
888:	learn: 281936.8806134	total: 11.6s	remaining: 1.45s
889:	learn: 281813.2940340	total: 11.7s	remaining: 1.44s
890:	learn: 281720.8111362	total: 11.7s	remaining: 1.43s
891:	learn: 281603.0049780	total: 11.7s	remaining: 1.41s
892:	learn: 281489.9892289	total: 11.7s	remaining: 1.4s
893:	learn: 281398.1313208	total: 11.7s	remaining: 1.39s
894:	learn: 281231.3685845	total: 11.7s	remaining: 1.37s
895:	learn: 281142.1339167	total: 11.7s	remaining: 1.36s
896:	learn: 281067.4783867	total: 11.7s	remaining: 1.35s
897:	learn: 280952.8468912	total:

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [46:13<00:00, 693.41s/it]

Wall time: 46min 13s


In [20]:
super_model = RandomForestRegressor()
super_model.fit(stacked_features_train, y)

RandomForestRegressor()

In [21]:
mape(y, super_model.predict(stacked_features_train))

5.15177988942443

In [22]:
make_submit_file(super_model, stacked_features_test, "stacking_10.csv") # 15.89

## Выводы

Стэкинг - хорошая технология, очень интересно было ей воспользоваться, но результаты не очень.

# Усреднение результатов нескольких моделей

Давайте попробуем взять 3 последних модели (Случайный лес, catboost и xgboost), и усредним их результаты с коэффициентами (заданными вручную)

In [8]:
model_dt = pickle.load(open("5.2.random_forest_2000.model", "rb"))

In [14]:
model_cat = pickle.load(open("5.3.catboost.model", "rb"))

In [15]:
model_xg = pickle.load(open("5.4.xgboost.model", "rb"))

In [18]:
predict_dt = model_dt.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:   28.9s finished


In [19]:
predict_cat = model_cat.predict(X_test_cat)

In [20]:
prediict_xg = model_xg.predict(X_test)

In [25]:
predict_all = np.array([predict_dt, predict_cat, prediict_xg])
coefs = np.array([0.5, 0.2, 0.3])

In [36]:
pred = np.round((predict_all.T * coefs).sum(axis=1))

In [37]:
filname ="ansamble.csv"
answer = pd.Series(pred, index=df[~df.is_train].sell_id, name='price')
answer.to_csv(filname, index_label='sell_id')
#result 13.03

## Выводы

Усреднение дало результат хуже чем случайный лес, наша лучшая модель.  
Получилось среднее между результатами всех моделей

# Выводы по этой части

Поиск лучшей модели не дал существенного результата, но было интересно.  
Лучшая модель оказалась той же, что я использовал с самого первого бейзлайна **random forest**